In [108]:
# Three-Pass Protocol

In [109]:
import numpy as np
import sympy as sym
from IPython.display import display, Math
import random

In [110]:
sym.init_printing() # prety printing of math

# Three-Pass Protocol

During a recent research I encountered a nice cryptographic protocol named: the "Three-pass protocol", originaly developed by Adi Shamir.  
https://en.wikipedia.org/wiki/Three-pass_protocol

The protocol's logic is really nice, and, in this post, I am going to explain and demonstrate its basic form (there are further developments and evolutions for this protocol, please check the Wikipedia link above).

For the algorythm implementation I use Python withing JupyterLab.

Alice wishes to send a secret to Bob through a public communication channel without exchanging cryptographic keys with Bob beforehand.   
The shared secret may be the secret-message itself, or a shared secret key or a password to be used in an encryption mechanism for encryption of data in a later communication session.  
In the example here I use the shared secret as a shared password that is then used to encrypt and decrypt a secret message.    
  
The idea of this protocol is somewhat like the following scenario:   
Alice and Bob are students sitting in the same classroom. This is a classroom of students from different faculties (where many of them don’t know each other). 
Alice wants to pass a secret message to Bob who sits on the other side of the class hall.   
Alice writes her secret message on a note, puts the note in a small box, and locks the box with her padlock.   
She writes "To Bob" on the box and asks the other students to pass the box to Bob.   
When Bob receives the box, he adds his padlock on the box, write on it "To Alice" and asks the students to pass the box back to Alice.  
Alice then receives the box, removes her padlock, writes on the box "To Bob" and asks the students to pass the box again.  
At this point, when Bob receives the box, he removes his padlock, opens the box, and read the secret message.  


Take note that this is an unauthenticated protocol. No sender nor responder have trust on the authenticity of the other side.   
In our example, the box may reach Mallory who is also in the class.  
Since people don’t know each other in this class, Mallory, who sits on the far side from Alice and has received the box, may put his padlock on the box instead of passing it to the person near him.  
Alice can’t see who received the box, and she believes that she is communicating with Bob, while she is communicating with Mallory.

In the scenario above Bob puts his padlock on the box near Alice’s, so Alice then can remove her padlock while Bob’s padlock is kept locked.    
In the three-pass protocol we put a second encryption (encryption == locked padlock) on top of the first one.    
It is as if Bob takes Alice’s locked box, puts it inside his box and locks his box using his padlock.    
For this protocol to work, it must be possible to remove the first encryption after a second encryption has been performed.    
Or in the scenario above, unlock Alice’s box while it is inside Bob’s locked box.  
In the Three-Pass protocol this is enabled through the use of ***modular multiplicative inverse***     

Cool!
     
     
*ka* - Alice's key   
*ka<sup>-1</sup>* - modular multiplicative inverse of *ka*    
*kb* - Bob's key     
*m* - Message   
*E* - Encryption    
*D* - Decryption    
            
*D(ka<sup>-1</sup>, E(kb, E(ka,m) ) ) = E(kb,m)*
        




This is possible with a commutative encryption, which is an encryption that is order-independent  
*E(ka, E(kb,m) ) = E(kb,E (ka,m) )*   
And then:   
*D(ka<sup>-1</sup>, E(kb, E(ka,m) ) ) = D(ka<sup>-1</sup>, E(ka, E(kb,m) ) ) = E(kb,m)*    

 Here is how this works with modular multiplicative inverse:

In [111]:
display( Math('k^{-1}k \cong b^{-1}b \cong 1(mod \\enspace p-1)') )

<IPython.core.display.Math object>

In [112]:
display( Math('k^{aba^{-1}b^{-1}} \cong k^1 \cong k(mod \\enspace p)') )

<IPython.core.display.Math object>

# The algorithm's steps

Shared secret key: ***ks***  
Alice's keys: ***ka***, ***ka<sup>-1</sup>***  
Bob's keys: ***kb***, ***kb<sup>-1</sup>***

1. Decide on numbers bits length (e.g., 128bits).
2. Everyone agree on a prime number *p* as a public key
3. Each user selects a random number *k*, that is a coprime to *p-1*, and *k*'s modular multiplicative inverse *k<sup>-1</sup>*   
    Alice's keys: *ka, ka<sup>-1</sup>*   
    Bob's keys:   *kb kb<sup>-1</sup>*   
4. Alise chooses a shared-secret-key *ks* to be shared with Bob
5. Alice encrypts a message using the shared secret: *c= E(ks,m)*
6. Alice sends *k1 $\cong$ ks<sup>ka</sup>(mod p)* to Bob.
7. Bob sends *k2 $\cong$ k1<sup>kb</sup>(mod p)* to Alice.
8. Alice sends *k3 $\cong$ k2<sup>ka<sup>-1</sup></sup>(mod p)* to Bob.
9. Bob computes *ks' $\cong$ k3<sup>kb<sup>-1</sup></sup>(mod p)*.
10. *assert ks' == ks* : verify that Bob's and Alice's shared-secret-keys are identical
11. Bob decrypts alice's encrypted message using the shared secret key he computed: *m'= D(ks',c)*
12. *assert m' == m* : verify that the original and received messages are identical

# Set global variables

#### Set symbolic variables

In [113]:
x,xi,y= sym.symbols('x xi y')

In [114]:
display( Math('Symbolic \\: variables: \\: \\textit{x, xi, y}') )

<IPython.core.display.Math object>

#### Set global variables

In [115]:
bits_length= 128 + 1   # bits length for the numbers: 2^bits-length
p= 0                   # public prime number

#### Define users

In [116]:
users= {'Alice':{'k':0, 'ki':0, 'ks':0}, 'Bob':{'k':0, 'ki':0, 'ks':0}}

# Functions

### 1. Choose a bits-length for the numbers

In [117]:
def set_globals(_bits_length:int=129, _p:int=0):
    global bits_length, p
    bits_length= _bits_length
    p= _p

### 2. Everyone agree on a random generated public key: *p* (with the required bits length)

In [118]:
# find the min and max ints with a defined bits length

def get_min_max_for_prime() -> tuple([int,int]):
    global bits_length
    
    min_int= int((2**bits_length / 2))  # Smallest int with the define length of bits
    max_int= (2**bits_length) - 1       # Largest int with the define length of bits
    
    return min_int, max_int

# for 128 bits:
# min= 170141183460469231731687303715884105728
# max= 340282366920938463463374607431768211455

In [119]:
# max_int_1= (2**bits_length)       # Largest int with the define length of bits
# display( Math('Max \\: number + 1: %i , \\: with \\: %i \\: bits' %(max_int_1, max_int_1.bit_length())) )

In [120]:
# Get a random prime number in the range of min_int and max_int
# randprime(a, b) - returns a random prime number in the range [a, b).

# Range max is not inclusive, so I add 1, which makes it longer in one bit (e.g., 129 bits instead of 128 bits), 
# but since it is not inclusive, the max number is the right number of bits
# 𝑀𝑎𝑥 𝑛𝑢𝑚𝑏𝑒𝑟 + 1 : 340282366920938463463374607431768211456, W𝑖𝑡ℎ 129 𝑏𝑖𝑡𝑠

def generate_prime():
    
    global p
    
    min_int, max_int = get_min_max_for_prime()
    # display( Math('Min \\: number: %i , \\: with \\: %i \\: digits' %(min_int, len(str(min_int)))) )
    # display( Math('Max \\: number: %i , \\: with \\: %i \\: digits' %(max_int, len(str(max_int)))) )

    p= sym.randprime(min_int, max_int+1)

    assert sym.isprime(p), "%i , is not a prime!" %p
    display( Math('Prime \\: number: %i , \\: with \\: %i \\: bits' %(p, p.bit_length())) )

In [121]:
# Use Crypto lib for generating the random prime number
# #!pip install pycryptodome

# from Crypto.Random import get_random_bytes
# from Crypto.Util.number import getPrime

# p= getPrime(bits_length, randfunc= get_random_bytes)
# display(p)
# print(len(str(p)))

### 3. Each user selects a random number *k* that is coprime to *p-1* and its modular multiplicative inverse *k<sup>-1</sup>*

#### 3.1. Find a random number *k* that is coprime to *p-1*

In [122]:
 display( Math('gcd(k,p-1)=1') )

<IPython.core.display.Math object>

In [123]:
def generate_k_for_users():
    # display( Math('gcd(k,p-1)=1') )
    
    global users, p, bits_length
        
    for user in users.keys():
        while( sym.gcd(users[user]['k'],p-1) != 1 ):
            users[user]['k']= random.getrandbits(bits_length)

    for user in users.keys():
        #display( Math('%s \\: k= %i' %(user, users[user]['k'])) )
        assert sym.gcd(x,y).subs({x:users[user]['k'], y:(p-1)}) == 1,  \
               "%s private k, is not coprime to p-1!" %user    

#### 3.2 Find *k<sup>-1</sup>*, the modular multiplicative inverse of *k*

In [124]:
display( Math('k^{-1}k \cong 1(mod \\enspace p-1)') )

<IPython.core.display.Math object>

In [125]:
def generate_k_inverse_for_users():
    # SymPy mod_inverse
    # https://docs.sympy.org/latest/modules/core.html#sympy.core.numbers.mod_inverse

    # display( Math('k^{-1}a(mod \\enspace p-1)') )
    
    global users, p
    
    for user in users.keys():
        users[user]['ki']= sym.mod_inverse( users[user]['k'], p-1) # ki: k-inverse

    # k*ki(mod p-1) = 1(mod p-1)
    for user in users.keys():
        k= users[user]['k']
        ki= users[user]['ki']
        assert sym.Mod(x*xi, y).subs({x:k, xi:ki, y:(p-1)}) == sym.Mod(1, p-1),  \
               "%s private k-inverse, is not coprime to p-1!" %user

In [126]:
def display_users_keys():
    global users
    
    display( Math('gcd(k,p-1)=1') )
    display( Math('k^{-1}k(mod \\enspace p-1)') )
    display( Math('k^{-1}k(mod \\enspace p-1) \\cong 1(mod \\enspace p-1)') )

    for user in users.keys():
        k= users[user]['k']
        ki= users[user]['ki']

        display( Math('%s \\: k= %i , \\: with \: %i \: bits' %(user, k, k.bit_length())) )
        display( Math('%s \\: k^{-1}= %i , \: with \: %i \: bits' %(user, ki, ki.bit_length())) )

### 4. Alise chooses a shared-secret-key *ks* to be shared with Bob

In [127]:
def alice_generate_shared_secret():
    
    global users, bits_length
    
    users['Alice']['ks']= random.getrandbits(bits_length-1)

### 5. Alice sends *k1 $\cong$ ks<sup>ka</sup>(mod p)* to Bob.

In [128]:
display( Math('k1 \cong ks^{ka}(mod \\enspace p)') )

<IPython.core.display.Math object>

In [129]:
# Modular Exponentiation
# a^b(mod p)
# sym.Mod seems to run forever
# Instead, using pow, see: https://docs.python.org/2/library/functions.html#pow
#d = pow(a, b, p) 

def alice_get_k1() -> int:
    global users, p
    
    k1= pow(users['Alice']['ks'],users['Alice']['k'],p)

    display( Math('\\text {Alice sends k1 to Bob.}') )
    display( Math('k1= %i , \\: with \: %i \: bits' %(k1, k1.bit_length())) )
    
    return k1

### 6. Bob sends *k2  $\cong$ k1<sup>kb</sup>(mod p)* to Alice.

In [130]:
display( Math('k2 \cong k1^{kb}(mod \\enspace p)') )

<IPython.core.display.Math object>

In [131]:
# k2= k1^kb(mod p)

def bob_get_k2(k1:int) -> int:
    global users, p
    
    k2= pow(k1,users['Bob']['k'],p)

    display( Math('\\text {Bob sends k2 to Alice.}') )
    display( Math('k2= %i , \\: with \: %i \: bits' %(k2, k2.bit_length())) )
    
    return k2

### 7. Alice sends *k3 $\cong$ k2<sup>ka<sup>-1</sup></sup>(mod p)* to Bob.

In [132]:
display( Math('k3 \cong k2^{ka^{-1}}(mod \\enspace p)') )

<IPython.core.display.Math object>

In [133]:
# k3= k2^(ka^-1)(mod p)

def alice_get_k3(k2:int) -> int:
    global users, p
    
    k3= pow(k2,users['Alice']['ki'],p)

    display( Math('\\text {Alice sends k3 to Bob.}') )
    display( Math('k3= %i , \\: with \: %i \: bits' %(k3, k3.bit_length())) )
    
    return k3

### 8. Bob computes *ks'= k3<sup>kb<sup>-1</sup></sup>(mod p)*.

In [134]:
display( Math("ks' \cong k3^{kb^{-1}}(mod \\enspace p)") )

<IPython.core.display.Math object>

In [135]:
# ks_2= k3^(kb^-1)(mod p)
# ks_2 is Bob's computed shared-key. It has to be identical to ks.

def bob_compute_shared_secret(k3:int):
    global users, p
    
    users['Bob']['ks']= pow(k3,users['Bob']['ki'],p)

### 9. verify the two shared secret keys, *ks and ks* are identical

In [136]:
def verify_shared_keys_indentical() -> bool:
    global users
    
    assert users['Bob']['ks'] == users['Alice']['ks'], "Secret shared keys are NOT indentical!"
    
    result= True
    if users['Bob']['ks'] != users['Alice']['ks']:
        result= False
        
    return result

# Use the shared secret for messaging

aes-cipher 2.0.0  
https://github.com/ebellocchia/aes_cipher

In [137]:
# ! pip install aes-cipher

In [138]:
from aes_cipher import DataEncrypter, Pbkdf2Sha512Default, DataDecrypter

### Alice encrypts a message using the shared key

In [139]:
# return the encrypted message cipher as <class 'bytes'>

def get_alice_encrypted_message(msg_data:str, shared_secret:int) -> bytes:
    data_encrypter = DataEncrypter(Pbkdf2Sha512Default)
    data_encrypter.Encrypt(msg_data, str(shared_secret))
    enc_data = data_encrypter.GetEncryptedData()
    #print(type(enc_data))
    
    return enc_data

### Bob decrypts Alice's message using the shared key

In [140]:
# return the decrypted cleartext message string

def get_decrypted_cleartext_message(msg_ciphertext_data:bytes, shared_secret:int) -> str:
    data_decrypter = DataDecrypter(Pbkdf2Sha512Default)
    data_decrypter.Decrypt(msg_ciphertext_data, str(shared_secret))
    dec_data = data_decrypter.GetDecryptedData()
    # display('Alice message cleartext: %s' %dec_data)
    
    return dec_data

In [141]:
# verify that the messages are identical

def verify_messages_indentical(aoriginal_msg:str,decrypted_msg:str) -> bool:
    result= False
    
    # remove the b and apostrophes from the decrypted message (b'a message from Alice')
    received_msg= str(decrypted_msg)[2:-1]
    # print(received_msg)
    
    if(received_msg == aoriginal_msg):
        result= True
        
    return result

## Execute

In [142]:

secret_msg= "a secret message from Alice"
display( Math('\\text{Alice original message: } \\text{"%s"}' %secret_msg))

# 1. Decide on numbers bits length: 2^bits-length
set_globals(_bits_length=129,_p=0)

# 2. Everyone agree on a public key: p, and publish it
generate_prime()

# 3. Each user selects a random number k that is a coprime to p-1 and its modular multiplicative inverse k-1
generate_k_for_users()
generate_k_inverse_for_users()

# 4. Alise chooses a shared-secret-key ks to be shared with Bob
alice_generate_shared_secret()
display( Math("\\text{Alice's shared secret = } %i , \\: with \: %i \: bits" %(users['Alice']['ks'], users['Alice']['ks'].bit_length())) )

# 5. Alice encrypts a message using the shared secret
alice_msg_cipher= get_alice_encrypted_message(msg_data=secret_msg, shared_secret=users['Alice']['ks'])
display( Math('\\text{Alice message cipher: } %s %s' %(str(alice_msg_cipher)[:20], ". . .")) )

# 6. Alice sends k1= kska(mod p) to Bob.
k1= alice_get_k1()

# 7. Bob sends k2= k1kb(mod p) to Alice.
k2= bob_get_k2(k1)

# 8. Alice sends k3= k2ka-1(mod p) to Bob.
k3= alice_get_k3(k2)

# 9. Bob computes kb= k3kb-1(mod p).
bob_compute_shared_secret(k3)
display( Math('\\text{Bob computes the shared secret.}') )
display( Math("\\text{Bob's shared secret = } %i , \\: with \: %i \: bits" %(users['Bob']['ks'], users['Bob']['ks'].bit_length())) )

# 10. verify: assert(alice_ks == bob_ks)
if verify_shared_keys_indentical() == True:
    display( Math("\\text{Secret shared keys are identical!}") )
    
# 11. Bob decrypts alice's encrypted message using the shared secret key he computed
decrypted_msg= get_decrypted_cleartext_message(msg_ciphertext_data=alice_msg_cipher, shared_secret=users['Bob']['ks'])

# 12. verify that the original and received messages are identical
display( Math("\\text{Alice's decrypted message by Bob: } \\text{'%s'}" %str(decrypted_msg)[2:-1]))
if verify_messages_indentical(secret_msg,decrypted_msg) == True:
    display(Math("\\text{The original and decrypted messages are identical!}") )


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>